In [1]:
import ee
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="ee-bau-tubitak-ndvi")
image_data = ee.ImageCollection("COPERNICUS/S2_SR")
parsel_data = ee.FeatureCollection("projects/ee-bau-tubitak-ndvi/assets/DestekParselleri")
roi = parsel_data.geometry().convexHull()

In [ ]:
start_date = ee.Date.fromYMD(2018, 1, 1)
end_date = start_date.advance(1, "month")


image = ee.ImageCollection("COPERNICUS/S2") \
    .filterDate(start_date, end_date) \
    .median()

image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1],
}

Map = geemap.Map()
Map.add_layer(image, image_viz_params, 'false color composite')

Map